In [ ]:
!pip install PySastrawi

In [ ]:
import json
import pandas as pd 
import numpy as np

# Data Loading

In [ ]:
#data1 keyword "kurir cod" 1 Juni-14 Juni
#data2 keyword "kurir cod" 1 Mei-31 Mei
#data3 keyword "sistem cod shopee" 1 Mei-14 Juni
#data4 keyword "sistem cod tokopedia" 1 Mei-14 Juni
#data5 keyword "sistem cod lazada" 1 Mei-14 Juni
#data6 keyword "sistem cod marketplace" 1 Mei-14 Juni
#data7 keyword "cod @shopeeid" 1 Mei-14 Juni
#data8 keyword "hapus cod" 1 Mei-14 Juni
#data9 keyword "@bukuakik" 1 Mei-14 Juni

df1 = pd.read_json(r'../input/raw-data-tweet-cod/data1.json',lines=True)
df2 = pd.read_json(r'../input/raw-data-tweet-cod/data2.json',lines=True)
df3 = pd.read_json(r'../input/raw-data-tweet-cod/data3.json',lines=True)
df4 = pd.read_json(r'../input/raw-data-tweet-cod/data4.json',lines=True)
df5 = pd.read_json(r'../input/raw-data-tweet-cod/data5.json',lines=True)
df6 = pd.read_json(r'../input/raw-data-tweet-cod/data6.json',lines=True)
df7 = pd.read_json(r'../input/raw-data-tweet-cod/data7.json',lines=True)
df8 = pd.read_json(r'../input/raw-data-tweet-cod/data8.json',lines=True)

In [ ]:
df1 = df1.where(pd.notnull(df1), None)
df2 = df2.where(pd.notnull(df2), None)
df3 = df3.where(pd.notnull(df3), None)
df4 = df4.where(pd.notnull(df4), None)
df5 = df5.where(pd.notnull(df5), None)
df6 = df6.where(pd.notnull(df6), None)
df7 = df7.where(pd.notnull(df7), None)
df8 = df8.where(pd.notnull(df8), None)

In [ ]:
df1.info()

In [ ]:
def extractTweets(df, df_source):
    for i in range(len(df_source)):

        tweet_date = df_source['date'][i]
        content = df_source['content'][i]
        tweet_id = df_source['id'][i]
        user_id = df_source['user'][i]['id']
        user_username = df_source['user'][i]['username']
        user_is_verified = df_source['user'][i]['verified']
        user_created = df_source['user'][i]['created']
        user_followers = df_source['user'][i]['followersCount']
        user_statuses_count = df_source['user'][i]['statusesCount']
        tweet_reply_count = df_source['replyCount'][i]
        tweet_retweet_count = df_source['retweetCount'][i]
        tweet_like_count = df_source['likeCount'][i]
        tweet_lang = df_source['lang'][i]
        quoted_tweet_id = df_source['quotedTweet'][i]
        replied_tweet_id = df_source['inReplyToTweetId'][i]

        if(df_source['inReplyToUser'][i] != None):
            replied_user = df_source['inReplyToUser'][i]['id']
        else:
            replied_user = None

        mentioned_users_id = []
        mentioned_users_username = []
        if(df_source['mentionedUsers'][i] != None):
            for mentioned_users in df_source['mentionedUsers'][i]:
                mentioned_users_id.append(mentioned_users['id'])
                mentioned_users_username.append(mentioned_users['username'])
        else: 
            mentioned_users_id = None
            mentioned_users_username = None

        if(df_source['place'][i] != None):
            tweet_place = df_source["place"][i]['name']
            tweet_country = df_source["place"][i]['country']
        else: 
            tweet_place = None
            tweet_country = None

        a_row = pd.Series([tweet_date, content, tweet_id, user_id, user_username, user_is_verified, user_created, user_followers, user_statuses_count, tweet_reply_count, tweet_retweet_count, tweet_like_count, tweet_lang, quoted_tweet_id, replied_tweet_id, replied_user, mentioned_users_id, mentioned_users_username, tweet_place, tweet_country])
        row_df = pd.DataFrame([a_row], index = [tweet_id])
        df = pd.concat([df, row_df])
    
    index = df.index
    is_duplicate = index.duplicated(keep="first")
    not_duplicate = ~is_duplicate
    df = df[not_duplicate]
    
    print(sum(is_duplicate),"data with non-unique indices deleted")
    print(sum(not_duplicate),"data successfully extracted")
    print("total data: ", len(df))
    print("\n")
    
    return df

In [ ]:
#del tweets_df

In [ ]:
tweets_df = pd.DataFrame()
tweets_df = extractTweets(tweets_df, df1)
tweets_df = extractTweets(tweets_df, df2)
tweets_df = extractTweets(tweets_df, df3)
tweets_df = extractTweets(tweets_df, df4)
tweets_df = extractTweets(tweets_df, df5)
tweets_df = extractTweets(tweets_df, df6)
tweets_df = extractTweets(tweets_df, df7)
tweets_df = extractTweets(tweets_df, df8)

tweets_df = tweets_df.rename(columns={0: "tweet_date", 
                                        1: "content", 
                                        2: "tweet_id", 
                                        3: "user_id",
                                        4: "user_username",
                                        5: "user_is_verified",
                                        6: "user_created",
                                        7: "user_statuses_count",
                                        8: "user_followers",
                                        9: "tweet_reply_count",
                                        10: "tweet_retweet_count",
                                        11: "tweet_like_count",
                                        12: "tweet_lang",
                                        13: "quoted_tweet_id",
                                        14: "replied_tweet_id",
                                        15: "replied_user",
                                        16: "mentioned_users_id",
                                        17: "mentioned_users_username",
                                        18: "tweet_place",
                                        19: "tweet_country"}, errors="raise")
tweets_df = tweets_df.reset_index(drop=True)
tweets_df

In [ ]:
tweets_df

In [ ]:
alay_dict = pd.read_csv("../input/indonesian-abusive-and-hate-speech-twitter-text/new_kamusalay.csv", encoding = 'latin-1', header=None)
alay_dict = alay_dict.rename(columns=  {0: 'original',
                                        1: 'replacement'})
stopword_dict = pd.read_csv('../input/indonesian-stoplist/stopwordbahasa.csv', header=None)
stopword_dict = stopword_dict.rename(columns={0: 'stopword'})

# Pre-processing

In [ ]:
from datetime import datetime
import pytz

In [ ]:
def convert_date(dt_object):
    tz = pytz.timezone("Asia/Jakarta")
    local_dt = dt_object.astimezone(tz)
    return local_dt.strftime("%d %b %Y")

In [ ]:
tweets_df["tweet_date"] = tweets_df["tweet_date"].apply(convert_date)
print(tweets_df["tweet_date"])

In [ ]:
import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#lowercase
def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
    text = re.sub('\n',' ',text) # Remove every '\n'
    text = re.sub('rt',' ',text) # Remove every retweet symbol
    text = re.sub('((@[^\s]+)|(#[^\s]+))',' ',text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    text = re.sub('  +', ' ', text) # Remove extra spaces
    return text
    
def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text

alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

def remove_stopword(text):
    text = ' '.join(['' if word in stopword_dict.stopword.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = text.strip()
    return text

def stemming(text):
    return stemmer.stem(text)

In [ ]:
#function test
testing_tweet = 'wkwkw berwawasan DUNIAWI abis RT @_rizkaAnggie "The quick brown fox jumps over the lazy dog" www.wikipedia.org #mantap_gan'


print(testing_tweet, "\t (original tweet)")
testing_tweet = lowercase(testing_tweet)
print(testing_tweet, "\t (lowercase)")
testing_tweet = remove_unnecessary_char(testing_tweet)
print(testing_tweet, "\t (remove_unnecessary_char)")
testing_tweet = remove_nonaplhanumeric(testing_tweet)
print(testing_tweet, "\t (remove_nonaplhanumeric)")
testing_tweet = normalize_alay(testing_tweet)
print(testing_tweet, "\t (normalize_alay)")
testing_tweet = remove_stopword(testing_tweet)
print(testing_tweet, "\t (remove_stopword)")
testing_tweet = stemming(testing_tweet)
print(testing_tweet, "\t (stemming)")

In [ ]:
def preprocess(text):
    text = lowercase(text) # 1
    text = remove_unnecessary_char(text) # 2
    text = remove_nonaplhanumeric(text) # 3
    text = normalize_alay(text) # 4
    text = remove_stopword(text) # 5
    text = stemming(text) # 6
    return text

In [ ]:
print(tweets_df['content'][2])
print(preprocess(tweets_df['content'][2]))

In [ ]:
clean_tweets_df = pd.DataFrame()
clean_tweets_df['tweet_id'] = tweets_df['tweet_id']
clean_tweets_df['tweet'] = tweets_df['content'].apply(preprocess)

In [ ]:
clean_tweets_df

# wordcloud

In [ ]:
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt

In [ ]:
text = " ".join(review for review in clean_tweets_df.tweet)
print ("There are {} words in the combination of all review.".format(len(text)))

In [ ]:
stopwords = set(STOPWORDS)
stopwords.update(["nya", "ya", "kayak", "sih", "deh", "yang", "biar"])

wordcloud = WordCloud(stopwords=stopwords, max_font_size=100, max_words=200, background_color="black", width=1200, height=800, colormap="Blues").generate(text)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
wordcloud.to_file("wordcloud1.png")

# Sentiment Analysis

In [ ]:
import csv

In [ ]:
lexicon_positive = dict()
with open('../input/inset-lexicon/positive.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        lexicon_positive[row[0]] = int(row[1])

lexicon_negative = dict()
with open('../input/inset-lexicon/negative.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        lexicon_negative[row[0]] = int(row[1])

In [ ]:
def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:
    score = 0
    for word in text:
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
    for word in text:
        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
    polarity=''
    if (score > 0):
        polarity = 'positive'
    elif (score < 0):
        polarity = 'negative'
    else:
        polarity = 'neutral'
    return score, polarity

In [ ]:
clean_tweets_df

In [ ]:
tweet_words = pd.Series()
clean_tweets_df.insert(len(clean_tweets_df.columns), 'tweet_words', '')

for i, text in enumerate(clean_tweets_df['tweet']):
    list_words=[]
    for word in clean_tweets_df['tweet'][i].split():
        list_words.append(word)
        
    clean_tweets_df['tweet_words'][i] = list_words 
clean_tweets_df

In [ ]:
results = clean_tweets_df['tweet_words'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_tweets_df['polarity_score'] = results[0]
clean_tweets_df['polarity'] = results[1]
print(clean_tweets_df['polarity'].value_counts())

In [ ]:
clean_tweets_df["raw_tweet"] = tweets_df['content']

In [ ]:
pd.set_option('display.max_colwidth', 3000)
positive_tweets = clean_tweets_df[clean_tweets_df['polarity'] == 'positive']
positive_tweets = positive_tweets[['raw_tweet', 'polarity_score', 'polarity']].sort_values(by = 'polarity_score', ascending=False).reset_index(drop = True)
positive_tweets.index += 1
positive_tweets[0:10]

In [ ]:
negative_tweets = clean_tweets_df[clean_tweets_df['polarity'] == 'negative']
negative_tweets = negative_tweets[['raw_tweet', 'polarity_score', 'polarity']].sort_values(by = 'polarity_score', ascending=True).reset_index(drop = True)
negative_tweets.index += 1
negative_tweets[0:10]

# Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
clean_tweets_df["date"]  = clean_tweets_df.tweet_id.map(tweets_df.set_index('tweet_id')['tweet_date'])

In [ ]:
clean_tweets_df

In [ ]:
def groupbyPolarity(g):
    # do computation
    return g.groupby(['polarity']).count()

In [ ]:
df_count = clean_tweets_df[["date","polarity","tweet"]].groupby(['date','polarity']).count().reset_index()
df_count['date'] = pd.to_datetime(df_count.date)
df_count.sort_values(by='date')

In [ ]:
df_count = df_count.pivot(index="date", columns="polarity", values="tweet")
df_count.plot(figsize=(16,8))